In [11]:
import matplotlib.pyplot as plt
import pandas as p
import torch
import numpy as np

In [12]:
%run ./digits.py
# print(train_images, train_labels, test_images, test_labels)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [13]:
# convert to tensors TODO: validate whether this is the right way to do this
train_images = torch.tensor(train_images)
train_labels = torch.tensor(train_labels)
test_images = torch.tensor(test_images)
test_labels = torch.tensor(test_labels)

# show data shapes
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

torch.Size([60000, 28, 28]) torch.Size([60000]) torch.Size([10000, 28, 28]) torch.Size([10000])


In [ ]:
# Any data processing we need to do here?

In [ ]:
# tweakable hyperparameters for the model
# loss function and optimizer are placed in the training cell
batch_size = 25
num_of_epochs = 300
learning_rate = 0.005
neurons_per_layer = 128 # for the hidden layers

numbers_of_layers = 3 # num of hidden layers excl. the input and output layers
activation_function = torch.nn.Sigmoid() # try other activation functions too!

In [14]:
# the model
class DigitClassifier(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        
        self.all_layers = torch.nn.Sequential()

        # input layer
        self.all_layers.add_module('input', torch.nn.Linear(num_features, neurons_per_layer))
        self.all_layers.add_module('input_activation', activation_function)

        # hidden layers
        for i in range(numbers_of_layers):
            self.all_layers.add_module(f'hidden_{i}', torch.nn.Linear(neurons_per_layer, neurons_per_layer))
            self.all_layers.add_module(f'hidden_{i}_activation', activation_function)

        # output layer
        self.all_layers.add_module('output', torch.nn.Linear(neurons_per_layer, num_classes))
        self.all_layers.add_module('output_activation', torch.nn.Softmax(dim=1))

    def forward(self, x):
        return self.all_layers(x)        

In [ ]:
# Data loader
from torch.utils.data import Dataset, DataLoader

#TODO: implement our dataloader, the commented lines are suggested by Copilot
class DigitsDataset(Dataset):
    def __init__(self, images, labels):
        pass
        # self.images = images
        # self.labels = labels
    
    def __len__(self):
        pass
        # return len(self.images)
    
    def __getitem__(self, idx):
        pass
        # return self.images[idx], self.labels[idx]
    
train_dataset = DigitsDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# the training loop

import torch.nn.functional as F
torch.manual_seed(555)

# dimension n of the n*n input picture
n = 28

# number of classes (digits 0-9)
num_classes = 10

# the model
model = DigitClassifier(n*n, num_classes)

# loss and optimizer, try other combos too?
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# train!
num_epochs = num_of_epochs
epoch_losses = [] # to plot the loss curve later

for epoch in range(num_epochs):

    model = model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        logits = model(features)
        loss = F.mse_loss(logits, labels) # Loss function
        loss.backward()
        optimizer.step()
    
    # logging + save the loss
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}" f" | Batch {batch_idx:03d}/{len(train_loader):03d}" f" | Train/Val Loss: {loss:.2f}")
    epoch_losses.append(loss.item())


# save model after training
torch.save(model.state_dict(), './model.pth')

In [ ]:
# training loss plot
plt.plot(epoch_losses)
plt.legend(["Training Loss"])
plt.title("Training Loss vs. Number of Epoch")
plt.xlabel("Number of Epoch")
plt.ylabel("Training Loss")